In [1]:
import gym
import random
import math
import numpy as np

# CartPole

In [2]:
env = gym.make('CartPole-v0')

#### posible action : 
- left , right

In [3]:
env.action_space.n

2

#### Enviroment space : 
- position of cart
- velocity of cart
- angle of the pole to the verticle
- angular velocity in which the pole is moving

> low : give lower bounds of 4 values that make up the observation space (individual state variable cannot have values below these)

> high : give upper bounds of 4 values that make up the observation space

In [9]:
env.observation_space.low

array([-4.8000002e+00, -3.4028235e+38, -4.1887903e-01, -3.4028235e+38],
      dtype=float32)

## Discretize the state space :
> so we can apply Q_Learning to bounded space 
> (this is a technique to reduce dimentionality Q_value computation)

- position of cart (Left , Right)
> 1: ignoring this variable completely in our state space
- velocity of cart
> 1: ignoring this variable completely in our state space
- angle of the pole to the verticle
- angular velocity in which the pole is moving

In [11]:
## two first 1 refer to ignoring 2 dimention of state space
## 6 bucket refer to 6 discrete intervals
NUM_BUCKETS  = ( 1 , 1 , 6 , 3 )

In [12]:
NUM_ACTION   = env.action_space.n 

In [14]:
STATE_BOUNDS = list(zip(env.observation_space.low,env.observation_space.high))

### redefine some of bounds in order to further limit our state space

In [18]:
STATE_BOUNDS[1] = [-.5 , .5]
STATE_BOUNDS[3] = [-math.radians(50) , math.radians(50)]

STATE_BOUNDS

[(-4.8, 4.8),
 [-0.5, 0.5],
 (-0.41887903, 0.41887903),
 [-0.8726646259971648, 0.8726646259971648]]

### initialize Q_Table :

#### num_states × num_actions = (1,1,3,6)×2

In [27]:
q_table = np.zeros(NUM_BUCKETS+(NUM_ACTION,))
print("shape of Q_Table : {}\n".format(q_table.shape))
print(q_table)

### first 4 dimention refer to states
### last dimention refer to actions for each state

shape of Q_Table : (1, 1, 6, 3, 2)

[[[[[0. 0.]
    [0. 0.]
    [0. 0.]]

   [[0. 0.]
    [0. 0.]
    [0. 0.]]

   [[0. 0.]
    [0. 0.]
    [0. 0.]]

   [[0. 0.]
    [0. 0.]
    [0. 0.]]

   [[0. 0.]
    [0. 0.]
    [0. 0.]]

   [[0. 0.]
    [0. 0.]
    [0. 0.]]]]]


### we initially want to explore state space in order to fill up the Q-value in Q_Table
- we use an explanatory rate of 0

In [28]:
EXPLORATION_RATE_MIN = 0.01

In [29]:
LEARNING_RATE_MIN =  0.1